In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import numpy as np
#import datetime
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
import time
import re
#import schedule

In [2]:
# define a function to scrape the required data
def web_scraper_bcoin():
    page = requests.get("https://www.coinbase.com/price/bitcoin")
    soup = BeautifulSoup(page.content, 'html.parser')
    bit_price = soup.find_all(class_ = "ChartPriceHeader__BigAmount-sc-9ry7zl-4 dKeshi")
    price_string = bit_price[0].get_text()
    reg_string = re.findall(r"\d*\.\d{2}", price_string)# r - rowstring
    price = float(reg_string[0])
    bit_description = soup.find_all('h6', class_="Header__StyledHeader-sc-1q6y56a-0 gEArVu TextElement__Spacer-sc-18l8wi5-0 hpeTzd")
    bit_volume = bit_description[1].get_text()
    now = datetime.now()
    now = now.strftime("%d/%m/%Y %H:%M:%S")
    volume = float(bit_volume[1:5])
    df = pd.read_csv('price_every_min.csv', index_col = 0)
    df = df.append({'Date' : now , 'Price' : price, 'Volume': volume}, ignore_index=True)
    df.to_csv('price_every_min.csv')
# returns time, price and the volume

In [3]:
# Decision making
def price_watcher():
    df = pd.read_csv('price_every_min.csv', index_col = 0)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace = True)
    df1 = df['Price']
    short_window = 30
    mid_window = 60
    signals = pd.DataFrame(index=df1.index)# create a DataFrame
    #roll_d10 = df1.rolling(window=short_window).mean()
    roll_d10 = df1.ewm(span=short_window).mean()
    roll_d25 = df1.rolling(window=mid_window).mean()
    signals['short_mavg'] = roll_d10
    signals['mid_mavg'] = roll_d25
    signals['signal'] = (roll_d10 - roll_d25)
    def symbols(i):
        if i<0:
            return 0
        elif i>0:
            return 1
    #print(signals['signal'])
    signals['final_signal'] = signals['signal'].apply(symbols)
    #print(signals['final_signal'])
    signals['positions'] = signals['final_signal'].diff()
    df_signals = signals[['final_signal','positions']]
    df_signals.to_csv('live_signals.csv')
    #########
    plt.figure(figsize=(14, 7))
    plt.plot(df1.index, df1, lw=3, label='Original observations')
    plt.plot(df1.index, roll_d10, lw=3, label='Rolling mean (window small)')
    plt.plot(df1.index, roll_d25, lw=3, label='Rolling mean (window large)')

    plt.plot(signals.loc[signals.positions == 1.0].index, 
         signals.short_mavg[signals.positions == 1.0],
         '^', markersize=10, color='r', label='buy')
    plt.plot(signals.loc[signals.positions == -1.0].index, 
         signals.short_mavg[signals.positions == -1.0],
         'v', markersize=10, color='k', label='sell')
    plt.title('BTC-USD Price (The Technical Approach)')
    plt.tick_params(labelsize=12)
    plt.legend(loc='upper left', fontsize=12)
    plt.savefig('moving_avg')

In [4]:
## email sender
def email_sender():
    import pandas as pd
    import smtplib 
    from email.mime.multipart import MIMEMultipart 
    from email.mime.text import MIMEText 
    from email.mime.base import MIMEBase 
    from email import encoders 

    fromaddr = "sbn.adh@gmail.com"
    toaddr = "sbn.adh@gmail.com"

    # instance of MIMEMultipart 
    msg = MIMEMultipart() 

    # storing the senders email address 
    msg['From'] = 'sbn.adh@gmail.com'

    # storing the receivers email address 
    msg['To'] = 'sbn.adh@gmail.com'

    # storing the subject 
    msg['Subject'] = "Message from SABOT"
    signal = '0'
    df = pd.read_csv('live_signals.csv', index_col = 0)
    if df.positions.values[-1] == 1:
        signal = 'TIME TO BUY!'
    elif df.positions.values[-1] == -1:
        signal = 'TIME TO SELL!'
    # string to store the body of the mail 
    body = signal

    # attach the body with the msg instance 
    msg.attach(MIMEText(body, 'plain')) 

    # open the file to be sent 
    filename = "moving_avg.png"
    attachment = open(filename, "rb") 

    # instance of MIMEBase and named as p 
    p = MIMEBase('application', 'octet-stream') 

    # To change the payload into encoded form 
    p.set_payload((attachment).read()) 

    # encode into base64 
    encoders.encode_base64(p) 

    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 

    # attach the instance 'p' to instance 'msg' 
    msg.attach(p) 

    # creates SMTP session 
    s = smtplib.SMTP('smtp.gmail.com', 587) 

    # start TLS for security 
    s.starttls() 

    # Authentication 
    s.login(fromaddr, "movedtob13") 

    # Converts the Multipart msg into a string 
    text = msg.as_string() 
    if df.positions.values[-1] == 1 or df.positions.values[-1] == -1: 
        s.sendmail(fromaddr, toaddr, text) 

        # terminating the session 
        s.quit()        

In [ ]:
while True:
    web_scraper_bcoin()
    price_watcher()
    email_sender()
    time.sleep(600)

C:\Users\sabin\Anaconda3\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
